# Matematikai algoritmusok és felfedezések II. - Negyedik gyakorlat (SAT)

## SAT - Logikai kielégíthetőség

| Matematikai jelölés                     | Megnevezés             | PySAT szintaxis          |
| :------------------------------------- |    :----:              |                ---:      |
| $x_1$                                   | Literál                |       `1`                |
| $\lnot x_1$                             | Literál negáltja       |       `-1`               |
| $\left(x_1\lor x_2\lor\lnot x_3\right)$ | Klóz (clause)          |       `[1,2,-3]`         |
| $(x_1\lor x_2)\land(x_1\lor\lnot x_3)$  | Konjunktív normálforma | `CNF([[1, 2], [1, -3]])` |

Engedélyezett műveletek egy konjunktív normálformában (**CNF**):
* Literálok és tagadásuk "össze-**VAGY**-olása" klózokká,
* Klózok "össze-**ÉS**-elése".

Egy CNF ezen két művelet felhasználásával létrejövő logikai formula. A SAT feladat annak eldöntése, hogy tudunk-e olyan értéket adni a literáloknak (**IGAZ** vagy **HAMIS**), amelyek mellett a logikai formula **IGAZ**-ra értékelődik ki.

---

__Állítás:__ Tetszőleges logikai formula átalakítható konjunktív normálformára.

---

__Igazságtábla:__ Egy olyan táblázat, ahol felsoroljuk az összes lehetséges értékét a literáloknak ($n$ literál esetén ez $2^n$ lehetséges kiértékelés) illetve az adott értékek mellett a logikai formula értékét. Ennek segítségével tetszőleges logikai formula CNF-fé alakítható.

Például vegyük azt a kifejezést, hogy

$$\phi\colon = x_1 \land \lnot x_2\implies x_3 \land\lnot x_1.$$

Ez nyilvánvalóan nem CNF. Írjuk fel az igazságtábláját:

| $x_1$  | $x_2$  | $x_3$  | $\phi$ |
| :----: | :----: | :----: | :----: |
| $0$    | $0$    | $0$    | $1$    |
| $0$    | $0$    | $1$    | $1$    |
| $0$    | $1$    | $0$    | $1$    |
| $0$    | $1$    | $1$    | $1$    |
| $1$    | $0$    | $0$    | $0$    |
| $1$    | $0$    | $1$    | $0$    |
| $1$    | $1$    | $0$    | $1$    |
| $1$    | $1$    | $1$    | $1$    |

_(itt az egyszerűség kedvéért az **IGAZ** értéket $1$-gyel, a **HAMIS**-at $0$-val jelöltem.)_

Ez CNF-ként az alábbi lenne:

$$
    (\lnot x_1\lor x_2\lor x_3)\land(\lnot x_1\lor x_2\lor\lnot x_3).
$$

Az eljárás lényege:
* Kiválasztjuk azokat a sorokat, amelyekben **HAMIS** a formula értéke
* Ennek a sornak minden literálját tagadjuk, és csinálunk belőle egy klózt
* Az így kapott klózokat össze-**ÉS**-eljük.

## PySAT

In [2]:
from pysat.formula import CNF
from pysat.solvers import Glucose3
from pysat.formula import IDPool

In [14]:
# Create an empty CNF and add clauses
formula = CNF()
formula.append([1,2,3])
formula.append([-1,2,-3])

formula.clauses

[[1, 2, 3], [-1, 2, -3]]

In [15]:
formula = CNF()
formula.extend([[1,2,3], [-1,2,-3]])

formula.clauses

[[1, 2, 3], [-1, 2, -3]]

In [16]:
# Initialize solver instance, add clauses and solve it
g = Glucose3()
g.add_clause([1,2,3])
g.add_clause([-1,2,-3])
g.solve(), g.get_model()

(True, [1, -2, -3])

In [17]:
# Initialize solver and add clauses from a formula
g = Glucose3()
for c in formula.clauses:
    g.add_clause(c)

g.solve(), g.get_model()

(True, [1, -2, -3])

In [18]:
# Initialize solver with CNF and solve it
g = Glucose3(formula)

g.solve(), g.get_model()

(True, [1, -2, -3])

In [19]:
# Initialize solver with CNF and solve it with fixed variables
g = Glucose3(formula)

g.solve(assumptions=[2]), g.get_model()

(True, [-1, 2, -3])

In [20]:
# Variable IDs
vpool = IDPool()
x = lambda i: vpool.id(f'x_{i}')

formula = CNF()
for i in range(5):
    for j in range(i+1, 6):
        formula.append([x(i), x(j)])
        
print([[vpool.obj(i) for i in clause] for clause in formula.clauses])

[['x_0', 'x_1'], ['x_0', 'x_2'], ['x_0', 'x_3'], ['x_0', 'x_4'], ['x_0', 'x_5'], ['x_1', 'x_2'], ['x_1', 'x_3'], ['x_1', 'x_4'], ['x_1', 'x_5'], ['x_2', 'x_3'], ['x_2', 'x_4'], ['x_2', 'x_5'], ['x_3', 'x_4'], ['x_3', 'x_5'], ['x_4', 'x_5']]


## Telepítés

https://pysathq.github.io/index.html

__Telepítés után újra kell indítani a kernelt!__

In [21]:
!pip install --user python-sat

## Feladatok

### 1) Bemelegítés
Legyen $x_1,x_2$ és $x_3$ logikai változók, írj fel egy olyan $\phi(x_1,x_2,x_3)$ logikai formulát, amely pontosan akkor igaz, ha
* legfeljebb kettő változó igaz,
* legfeljebb egy változó igaz,
* minden változó igaz,
* pontosan az egyik változó igaz.

In [4]:
at_most_two = CNF()

at_most_two.extend([[-1,-2,-3]])
truth_table(at_most_two)

[1, 2, 3] False
[-1, 2, 3] True
[1, -2, 3] True
[1, 2, -3] True
[-1, -2, 3] True
[-1, 2, -3] True
[1, -2, -3] True
[-1, -2, -3] True


In [23]:
at_most_one = CNF()

at_most_one.extend([[-1,-2,-3],
                    [-1,-2,3],
                    [1, -2, -3],
                    [-1, 2, -3]])

truth_table(at_most_one)

[1, 2, 3] False
[-1, 2, 3] False
[1, -2, 3] False
[1, 2, -3] False
[-1, -2, 3] True
[-1, 2, -3] True
[1, -2, -3] True
[-1, -2, -3] True


In [6]:
same = CNF()

same.extend([[-1, -2, 3],
             [-1, 2, -3],
             [1, -2, -3],
             [-1, 2, 3],
             [1, -2, 3],
             [1, 2, -3],
             [1, 2, 3]])
truth_table(same)

[1, 2, 3] True
[-1, 2, 3] False
[1, -2, 3] False
[1, 2, -3] False
[-1, -2, 3] False
[-1, 2, -3] False
[1, -2, -3] False
[-1, -2, -3] False


In [25]:
g = Glucose3(same)
SAT = g.solve(assumptions=[1])
SAT, g.get_model()

(True, [1, 2, 3])

In [26]:
exactly_one = CNF()

exactly_one.extend([[1,2,3],
                    [-1,-2,-3],
                    [1,-2,-3],
                    [-1,-2,3],
                    [-1,2,-3]])
truth_table(exactly_one)

[1, 2, 3] False
[-1, 2, 3] False
[1, -2, 3] False
[1, 2, -3] False
[-1, -2, 3] True
[-1, 2, -3] True
[1, -2, -3] True
[-1, -2, -3] False


### 2) "Pontosan egy", "legfeljebb egy" segédfüggvények
Írj egy függvényt, amely klózoknak egy olyan listáját adja vissza, amely CNF-ként értelmezve pontosan akkor igaz, ha pontosan egy literál igaz/ legfeljebb egy literál igaz.

_(Hint: az egyiket felhasználva könnyen megkapható a másik.)_

In [13]:
def at_most_one(lits):
    '''
        Given a list of literals the function creates the clauses of the CNF
        that is true iff at most one literal is true
    '''
    clauses = []
    for i in range(len(lits)):
        for j in range(i+1, len(lits)):
            clauses.append([-lits[i], -lits[j]])
    return clauses

In [8]:
formula = CNF(from_clauses=at_most_one([1, 2, 3]))
truth_table(formula)

[1, 2, 3] False
[-1, 2, 3] False
[1, -2, 3] False
[1, 2, -3] False
[-1, -2, 3] True
[-1, 2, -3] True
[1, -2, -3] True
[-1, -2, -3] True


In [14]:
def exactly_one(lits):
    '''
        Given a list of literals the function creates the clauses of the CNF
        that is true iff only one literal is true
    '''
    clauses = at_most_one(lits)
    clauses.append(lits)
    return clauses

In [30]:
formula = CNF(from_clauses=exactly_one([1, 2, 3]))
truth_table(formula)

[1, 2, 3] False
[-1, 2, 3] False
[1, -2, 3] False
[1, 2, -3] False
[-1, -2, 3] True
[-1, 2, -3] True
[1, -2, -3] True
[-1, -2, -3] False


### 3) Igazságtábla

Írj egy függvényt, amelynek bemenete egy CNF, és kiírja a kapott CNF igazságtábláját!

In [3]:
import itertools

def truth_table(cnf):
    '''
        Compute all possible variations of truth values of literals,
        evaluate the formula and print the truth table
    '''
    g = Glucose3(cnf)
    lits = list(set(lit if lit > 0 else -lit for clause in cnf.clauses for lit in clause))
    for L in range(0, len(lits)+1):
        for subset in itertools.combinations(lits, L):
            assumption = [-i if i in subset else i for i in lits]
            print(assumption, g.solve(assumptions=assumption))

In [32]:
truth_table(same)

[1, 2, 3] True
[-1, 2, 3] False
[1, -2, 3] False
[1, 2, -3] False
[-1, -2, 3] False
[-1, 2, -3] False
[1, -2, -3] False
[-1, -2, -3] True


### 4) $n$-királynő probléma

Oldd meg az $n$-királynő problémát SAT feladatként! Az $n$-királynő problémában egy $n\times n$ méretű sakktáblára szeretnénk elhelyezni $n$ királynőt úgy, hogy semelyik kettő ne tudja ütni egymást.
* Modellezd SAT problémaként,
* oldd meg,
* ábrázold az így kapott sakktáblát

_(Hint: a korábban megírt függvények még hasznosak lehetnek)_

In [15]:
from pysat.card import CardEnc, EncType
def nqueens(n):
    '''
        Solve the n queens problem modeled as SAT
        input:
            n (int) : number of queens
            
        The pattern
    '''
    
    # Create an n x n board
    board = [[i+n*j for i in range(1, n+1)] for j in range(n)]
    
    # Initialize CNF
    nqueens = CNF()

    # Exactly one in any row
    for row in board:
        nqueens.extend(exactly_one(row))

    # Exactly one in any column
    for i in range(n):
        col = [row[i] for row in board]
        nqueens.extend(exactly_one(col))

    # At most one in any "right" diagonal
    rightdiags = []
    for k in range(0, n-1):
        i, j = k, 0
        rd = []
        while i < n and j < n:
            rd.append(board[j][i])
            i+= 1
            j+= 1
        rightdiags.append(rd)    

    for k in range(1, n-1):
        i, j = k, 0
        rd = []
        while i < n and j < n:
            rd.append(board[i][j])
            i+= 1
            j+= 1
        rightdiags.append(rd)

    for rd in rightdiags:
        nqueens.extend(at_most_one(rd))

    # At most one in any "left" diagonal
    leftdiags = []
    for k in range(n-1, 0, -1):
        i, j = 0, k
        ld = []
        while i < n and j >= 0:
            ld.append(board[i][j])
            i+=1
            j-=1
        leftdiags.append(ld)

    for k in range(n-1, 1, -1):
        i, j = k-1, n-1
        ld = []
        while i < n and j >= 0:
            ld.append(board[i][j])
            i+=1
            j-=1
        leftdiags.append(ld)

    for ld in leftdiags:
        nqueens.extend(at_most_one(ld))

    # Solve the constructed SAT
    solver = Glucose3(nqueens)
    solver.solve()
    
    # Retrieve solution
    sol = solver.get_model()
    return sol
        

In [16]:
import math
def print_nqueens(sol):
    '''
        Print solution of the n-queens problem
    '''
    n = int(math.sqrt(len(sol)))
    sep = "+" + n * "---+"
    print(sep)
    for i in range(n):
        row = "|"
        for j in range(n):
            if sol[i+j*n] > 0:
                row += " Q |"
            else:
                row += "   |"
        print(row)
        print(sep)

In [17]:
sol = nqueens(9)

In [18]:
print_nqueens(sol)

+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | Q |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   | Q |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   | Q |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   | Q |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
| Q |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   | Q |   |   |
+---+---+---+---+---+---+---+---+---+
|   | Q |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   | Q |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   | Q |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+


### 5) Sudoku
Oldd meg a Sudoku feladatot SAT problémaként! Hogyan kellene megoldani, ha adott előre néhány mező tartalma? 
* Modellezd SAT-ként,
* oldd meg,
* rajzold ki a kapott táblát!

__Ezen még gondolkodjatok, legkésőbb a hatodik gyakorlaton megnézzük!__